In [11]:
import sys
import os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist( normalize=True, one_hot_label=True)
print (x_train.shape)
print (t_train.shape)
t_train

(60000, 784)
(60000, 10)


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [12]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
print (batch_mask)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

[22316 21401  9324  3407 21217 11511 27997  7258 28722 55381]


In [13]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [14]:
y = np.random.rand(5, 10)
print(y.shape)
print(y)
t = [2, 7, 0, 9, 4]
y[np.arange(5), t]

(5, 10)
[[0.07371998 0.05721413 0.83573924 0.46702296 0.18586058 0.38084305
  0.85660625 0.43721048 0.77060427 0.21924388]
 [0.57125334 0.90383823 0.64370458 0.64596778 0.89435298 0.93069689
  0.24497463 0.48356606 0.55493827 0.33881976]
 [0.89635227 0.70470549 0.32181932 0.17499321 0.22695243 0.93701529
  0.84175895 0.16099354 0.42785679 0.75591977]
 [0.19992439 0.72547285 0.01878968 0.21287147 0.49443855 0.9624492
  0.47451905 0.28710109 0.27764998 0.10962917]
 [0.41479799 0.97322404 0.52054478 0.83723943 0.62630022 0.76220607
  0.53874582 0.27824557 0.62856802 0.60630715]]


array([0.83573924, 0.48356606, 0.89635227, 0.10962917, 0.62630022])

In [15]:
import sys
import os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3)

    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])

net = simpleNet()

In [16]:
net = simpleNet()
print(net.W)

[[ 0.39662651 -1.1566859  -0.83855752]
 [-0.051077    0.31150502  1.14714478]]


In [17]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)


[ 0.19200661 -0.41365702  0.52929579]


In [18]:
np.argmax(p)

np.int64(2)

In [19]:
t = np.array([0, 0, 1])
net.loss(x, t)


np.float64(0.7434495650129096)

In [20]:
x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])
def f(w):
    return net.loss(x, t)
dW = numerical_gradient(f, net.W)

print(dW)

[[ 0.20360676  0.11111068 -0.31471744]
 [ 0.30541014  0.16666603 -0.47207617]]
